In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import random

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import keras_tuner

2025-12-16 14:22:16.621267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from keras import layers, optimizers
import keras
from keras.optimizers import Adam, RMSprop
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):
    input_shape=(256, 256, 1)
    num_classes=28
    learning_rate=0.001
    model = keras.models.Sequential()
    # Couches de convolution
    model.add(layers.Conv2D(filters=hp.Int("units", min_value=32, max_value=512, step=32),kernel_size=(5, 5), padding='same', activation='relu', input_shape=input_shape))
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)))
    if hp.Boolean('MaxPool'):
        model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(filters=hp.Int("units", min_value=32, max_value=512, step=32),kernel_size=(3, 3), padding='same', activation=hp.Choice("activation", ["relu", "tanh"])))
    if hp.Boolean('MaxPool'):
        model.add(layers.MaxPooling2D((2, 2)))

    # Aplatissement et couches denses
    model.add(layers.GlobalAveragePooling2D())
    if hp.Boolean('Dropout1'):
        model.add(layers.Dropout(0.2))
    model.add(layers.Dense(hp.Int("units", min_value=32, max_value=512, step=32), activation='relu'))
    if hp.Boolean('Dropout2'):
        model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

        # Compilation
    if hp.Choice('optimizer', ['adam', 'rmsprop']) == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif hp.Choice('optimizer', ['adam', 'rmsprop']) == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)



    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model(keras_tuner.HyperParameters())
model.summary()

/home/edgar/.pyenv/versions/DataBass/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1765891343.197825  176811 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5580 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-12-16 14:22:29.489410: W external/local_xla/xla/service/gpu/llvm_gpu_backend/default/nvptx_libdevice_path.cc:41] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28)             │           924 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,060 (47.11 KB)

 Trainable params: 12,060 (47.11 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="/kaggle/working/SearchModels",
    project_name="Conv2DSearch",
)

PermissionDeniedError: /kaggle; Permission denied

In [6]:
path_data = '/kaggle/input/bassspectrogram/spectrograms/spectrograms/B'

import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

file_paths = []
labels = []
for label in sorted(os.listdir(path_data)):
    label_path = os.path.join(path_data, label)
    if os.path.isdir(label_path):
        print(f"Indexing label: {label}")
        for file in sorted(os.listdir(label_path)):
            if file.endswith(".npy"):
                file_paths.append(os.path.join(label_path, file))
                labels.append(label)

print(f"Indexed {len(file_paths)} spectrogram files.")

Indexing label: A1
Indexing label: A2
Indexing label: Asharp1
Indexing label: Asharp2
Indexing label: B1
Indexing label: B2
Indexing label: C2
Indexing label: C3
Indexing label: Csharp2
Indexing label: Csharp3
Indexing label: D2
Indexing label: D3
Indexing label: Dsharp2
Indexing label: Dsharp3
Indexing label: E1
Indexing label: E2
Indexing label: E3
Indexing label: F1
Indexing label: F2
Indexing label: F3
Indexing label: Fsharp1
Indexing label: Fsharp2
Indexing label: Fsharp3
Indexing label: G1
Indexing label: G2
Indexing label: G3
Indexing label: Gsharp1
Indexing label: Gsharp2
Indexed 41052 spectrogram files.


In [7]:
# Step 2: Create a list of indices and shuffle them
indices = list(range(len(file_paths)))
random.shuffle(indices)

# Step 3: Use shuffled indices to reorder both lists simultaneously
shuffled_file_paths = [file_paths[i] for i in indices]
shuffled_labels = [labels[i] for i in indices]

print(f"Shuffled {len(shuffled_file_paths)} files. First 3 samples:")
for i in range(3):
    print(f'BEFORE : {file_paths[i]} > {labels[i]}')
    print(f"AFTER :  {shuffled_file_paths[i]} → {shuffled_labels[i]}")

Shuffled 41052 files. First 3 samples:
BEFORE : /kaggle/input/bassspectrogram/spectrograms/spectrograms/B/A1/B11-33105-1111-00006.npy > A1
AFTER :  /kaggle/input/bassspectrogram/spectrograms/spectrograms/B/Fsharp2/B13-42209-2231-15103.npy → Fsharp2
BEFORE : /kaggle/input/bassspectrogram/spectrograms/spectrograms/B/A1/B11-33105-1121-11862.npy > A1
AFTER :  /kaggle/input/bassspectrogram/spectrograms/spectrograms/B/B1/B22-35202-3341-17072.npy → B1
BEFORE : /kaggle/input/bassspectrogram/spectrograms/spectrograms/B/A1/B11-33105-1122-12486.npy > A1
AFTER :  /kaggle/input/bassspectrogram/spectrograms/spectrograms/B/Asharp1/masked_B12-34106-4413-07547.npy → Asharp1


In [8]:
from tensorflow.keras.utils import Sequence

class SpectrogramSequence(Sequence):
    def __init__(self, shuffled_file_paths, shuffled_labels, batch_size, num_classes):
        self.file_paths = np.array(shuffled_file_paths)
        self.labels = np.array(shuffled_labels, dtype=np.int32)
        self.batch_size = batch_size
        self.num_classes = num_classes

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        batch_paths = self.file_paths[batch_slice]
        batch_labels = self.labels[batch_slice]
        batch_x = []
        for path in batch_paths:
            spec = np.load(path)
            if spec.ndim == 2:
                spec = np.expand_dims(spec, axis=-1)
            batch_x.append(spec.astype(np.float32))
        batch_x = np.stack(batch_x, axis=0)
        batch_y = batch_labels.astype(np.int32)
        return batch_x, batch_y


In [ ]:
le = LabelEncoder()
le.fit(shuffled_labels)
encoded_labels = le.transform(shuffled_labels)
num_classes = len(le.classes_)
print(f"Number of classes: {num_classes}")
X_train, X_val, y_train, y_val = train_test_split(
    shuffled_file_paths, encoded_labels, test_size=0.2, stratify=encoded_labels, random_state=42
)
batch_size = 32
patience = 5

Number of classes: 28


In [13]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=patience-2,min_lr=1e-6,cooldown=0)
train_sequence = SpectrogramSequence(X_train, y_train, batch_size, num_classes)
val_sequence = SpectrogramSequence(X_val, y_val, batch_size, num_classes)
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
MaxPool (Boolean)
{'default': False, 'conditions': []}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
Dropout1 (Boolean)
{'default': False, 'conditions': []}
Dropout2 (Boolean)
{'default': False, 'conditions': []}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop'], 'ordered': False}


In [18]:
train_seq = SpectrogramSequence(X_train, y_train, batch_size=32, num_classes=num_classes)
val_seq = SpectrogramSequence(X_val, y_val, batch_size=32, num_classes=num_classes)

# Tuner search
tuner.search(
    train_seq,
    validation_data=val_seq,
    epochs=20,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
448               |448               |units
0.2               |0.2               |dropout
True              |True              |MaxPool
relu              |relu              |activation
False             |False             |Dropout1
False             |False             |Dropout2
adam              |adam              |optimizer

Epoch 1/20


I0000 00:00:1765891134.860585     109 service.cc:148] XLA service 0x79f31000a8b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765891134.861286     109 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1765891135.166891     109 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-12-16 13:19:07.227113: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng35{k2=4,k3=0} for conv (f32[32,448,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,448,128,128]{3,2,1,0}, f32[448,448,3,3]{3,2,1,0}, f32[448]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-1

KeyboardInterrupt: 

In [19]:
history = model.fit(
    train_sequence,
    validation_data=val_sequence,
    epochs=50,
    callbacks=[lr_scheduler,early_stopping]
)

model.save('/kaggle/working/databass_kaggle.keras')
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1765875213.910626     109 service.cc:148] XLA service 0x7b585c00cc00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765875213.911357     109 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1765875214.300283     109 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1765875226.381093     109 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1027/1027 ━━━━━━━━━━━━━━━━━━━━ 442s 416ms/step - accuracy: 0.1694 - loss: 2.6575 - val_accuracy: 0.7537 - val_loss: 0.9148 - learning_rate: 0.0010
Epoch 2/50
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 203s 197ms/step - accuracy: 0.5930 - loss: 1.0676 - val_accuracy: 0.8694 - val_loss: 0.4488 - learning_rate: 0.0010
Epoch 3/50
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 203s 197ms/step - accuracy: 0.7437 - loss: 0.6963 - val_accuracy: 0.9191 - val_loss: 0.3074 - learning_rate: 0.0010
Epoch 4/50
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 203s 197ms/step - accuracy: 0.8100 - loss: 0.5205 - val_accuracy: 0.9236 - val_loss: 0.2205 - learning_rate: 0.0010
Epoch 5/50
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 203s 197ms/step - accuracy: 0.8542 - loss: 0.4014 - val_accuracy: 0.9686 - val_loss: 0.1353 - learning_rate: 0.0010
Epoch 6/50
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 203s 198ms/step - accuracy: 0.8827 - loss: 0.3276 - val_accuracy: 0.9776 - val_loss: 0.1016 - learning_rate: 0.0010
Epoch 7/50
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 203s 197ms/step - acc

KeyboardInterrupt: 

In [9]:
path_data_g = '/kaggle/input/bassspectrogram/spectrograms/spectrograms/G'

file_paths_g = []
labels_g = []
for label in sorted(os.listdir(path_data_g)):
    label_path_g = os.path.join(path_data_g, label)
    if os.path.isdir(label_path_g):
        print(f"Indexing label: {label}")
        for file in sorted(os.listdir(label_path_g)):
            if file.endswith(".npy"):
                file_paths_g.append(os.path.join(label_path_g, file))
                labels_g.append(label)
print(f"Indexed {len(file_paths_g)} spectrogram files.")

Indexing label: A2
Indexing label: A3
Indexing label: A4
Indexing label: Asharp2
Indexing label: Asharp3
Indexing label: Asharp4
Indexing label: Ax2
Indexing label: Ax3
Indexing label: Ax4
Indexing label: B2
Indexing label: B3
Indexing label: B4
Indexing label: C3
Indexing label: C4
Indexing label: C5
Indexing label: Csharp3
Indexing label: Csharp4
Indexing label: Csharp5
Indexing label: Cx3
Indexing label: Cx4
Indexing label: Cx5
Indexing label: D3
Indexing label: D4
Indexing label: D5
Indexing label: Dsharp3
Indexing label: Dsharp4
Indexing label: Dsharp5
Indexing label: Dx3
Indexing label: Dx4
Indexing label: Dx5
Indexing label: E1
Indexing label: E2
Indexing label: E3
Indexing label: E4
Indexing label: E5
Indexing label: F2
Indexing label: F3
Indexing label: F4
Indexing label: Fsharp2
Indexing label: Fsharp3
Indexing label: Fsharp4
Indexing label: Fx2
Indexing label: Fx3
Indexing label: Fx4
Indexing label: G2
Indexing label: G3
Indexing label: G4
Indexing label: Gsharp2
Indexing la

In [10]:
# Step 2: Create a list of indices and shuffle them
indices_g = list(range(len(file_paths_g)))
random.shuffle(indices_g)

# Step 3: Use shuffled indices to reorder both lists simultaneously
shuffled_file_paths_g = [file_paths_g[i] for i in indices_g]
shuffled_labels_g = [labels_g[i] for i in indices_g]

print(f"Shuffled {len(shuffled_file_paths_g)} files. First 3 samples:")
for i in range(3):
    print(f'BEFORE : {file_paths_g[i]} > {labels_g[i]}')
    print(f"AFTER :  {shuffled_file_paths_g[i]} → {shuffled_labels_g[i]}")

Shuffled 41316 files. First 3 samples:
BEFORE : /kaggle/input/bassspectrogram/spectrograms/spectrograms/G/A2/G61-45105-1111-20598.npy > A2
AFTER :  /kaggle/input/bassspectrogram/spectrograms/spectrograms/G/D4/G63-62407-4412-38189.npy → D4
BEFORE : /kaggle/input/bassspectrogram/spectrograms/spectrograms/G/A2/G61-45105-1121-26838.npy > A2
AFTER :  /kaggle/input/bassspectrogram/spectrograms/spectrograms/G/C5/masked_G81-72608-2221-23474.npy → C5
BEFORE : /kaggle/input/bassspectrogram/spectrograms/spectrograms/G/A2/G61-45105-1122-27462.npy > A2
AFTER :  /kaggle/input/bassspectrogram/spectrograms/spectrograms/G/C4/masked_G61-60501-3322-30630.npy → C4


In [ ]:
le_g = LabelEncoder()
le_g.fit(shuffled_labels_g)
encoded_labels_g = le.transform(shuffled_labels_g)
num_classes_g = len(le_g.classes_)
print(f"Number of classes: {num_classes_g}")
X_traing, X_valg, y_traing, y_valg = train_test_split(
    shuffled_file_paths_g, encoded_labels_g, test_size=0.2, stratify=encoded_labels_g, random_state=42
)
batch_size = 32
train_sequence = SpectrogramSequence(X_traing, y_traing, batch_size, num_classes)
val_sequence = SpectrogramSequence(X_valg, y_valg, batch_size, num_classes)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

ValueError: y contains previously unseen labels: 'D4'

In [ ]:
history = model_g.fit(
    train_sequence,
    validation_data=val_sequence,
    epochs=50,
    callbacks=[early_stopping]
)

model.save('/kaggle/working/databass_kaggle_g.keras')
import pickle
with open('label_encoder.pkl', 'wb') as f_g:
    pickle.dump(le_g, f_g)